In [0]:
ratings_schema = "`UserID` INT, `MovieID` INT, `Rating` INT, `Timestamp` bigint"

In [0]:
# File location and type
ratings_file_location = "dbfs:/FileStore/tables/ratings.dat"

In [0]:
Ratings_DF=spark.read.format("csv").option("delimiter", "::").schema(ratings_schema).load(ratings_file_location)

In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
movies_schema = StructType([ 
    StructField("MovieID",IntegerType(),True), 
    StructField("Title",StringType(),True), 
    StructField("Genres",StringType(), True)])
movies_file_location = "dbfs:/FileStore/tables/movies.dat"
Movies_DF=spark.read.format("csv").option("delimiter", "::").schema(movies_schema).load(movies_file_location)
Final_Movies_DF=Movies_DF.withColumn("Genres",explode(split("Genres","[|]")))
Final_Movies_DF.show()

+-------+--------------------+---------+
MovieID| Title| Genres|
+-------+--------------------+---------+
 1| Toy Story (1995)|Adventure|
 1| Toy Story (1995)|Animation|
 1| Toy Story (1995)| Children|
 1| Toy Story (1995)| Comedy|
 1| Toy Story (1995)| Fantasy|
 2| Jumanji (1995)|Adventure|
 2| Jumanji (1995)| Children|
 2| Jumanji (1995)| Fantasy|
 3|Grumpier Old Men ...| Comedy|
 3|Grumpier Old Men ...| Romance|
 4|Waiting to Exhale...| Comedy|
 4|Waiting to Exhale...| Drama|
 4|Waiting to Exhale...| Romance|
 5|Father of the Bri...| Comedy|
 6| Heat (1995)| Action|
 6| Heat (1995)| Crime|
 6| Heat (1995)| Thriller|
 7| Sabrina (1995)| Comedy|
 7| Sabrina (1995)| Romance|
 8| Tom and Huck (1995)|Adventure|
+-------+--------------------+---------+
only showing top 20 rows

In [0]:
# What are the titles of top 5 most popular movies i.e. have the most ranking in the whole dataset?
joinDF=Final_Movies_DF.join(Ratings_DF,Final_Movies_DF.MovieID==Ratings_DF.MovieID).drop(Ratings_DF.MovieID)

In [0]:
joinDF.show()

+-------+--------------------+---------+------+------+---------+
MovieID| Title| Genres|UserID|Rating|Timestamp|
+-------+--------------------+---------+------+------+---------+
 122| Boomerang (1992)| Romance| 1| 5|838985046|
 122| Boomerang (1992)| Comedy| 1| 5|838985046|
 185| Net, The (1995)| Thriller| 1| 5|838983525|
 185| Net, The (1995)| Crime| 1| 5|838983525|
 185| Net, The (1995)| Action| 1| 5|838983525|
 231|Dumb & Dumber (1994)| Comedy| 1| 5|838983392|
 292| Outbreak (1995)| Thriller| 1| 5|838983421|
 292| Outbreak (1995)| Sci-Fi| 1| 5|838983421|
 292| Outbreak (1995)| Drama| 1| 5|838983421|
 292| Outbreak (1995)| Action| 1| 5|838983421|
 316| Stargate (1994)| Sci-Fi| 1| 5|838983392|
 316| Stargate (1994)|Adventure| 1| 5|838983392|
 316| Stargate (1994)| Action| 1| 5|838983392|
 329|Star Trek: Genera...| Sci-Fi| 1| 5|838983392|
 329|Star Trek: Genera...| Drama| 1| 5|838983392|
 329|Star Trek: Genera...|Adventure| 1| 5|838983392|
 329|Star Trek: Genera...| Action| 1| 5|838983392|
 355|Flintstones, The ...| Fantasy| 1| 5|838984474|
 355|Flintstones, The ...| Comedy| 1| 5|838984474|
 355|Flintstones, The ...| Children| 1| 5|838984474|
+-------+--------------------+---------+------+------+---------+
only showing top 20 rows

In [0]:
#What are the titles of top 5 most popular movies i.e. have the most ranking in the whole dataset?
from pyspark.sql.functions import desc
joinDF.groupBy("Title").agg(sum("Rating").alias("popularity")).filter(col("popularity").isNotNull()).sort(desc("popularity")).show(5)

+--------------------+----------+
 Title|popularity|
+--------------------+----------+
 Captain Ron (1992)| 2001284|
 Forrest Gump (1994)| 479352|
 Toy Story (1995)| 448135|
Jurassic Park (1993)| 414880|
 True Lies (1994)| 393775|
+--------------------+----------+
only showing top 5 rows

In [0]:
#What are the top 5 ranked movie genres on average in the whole dataset?
joinDF.groupBy("Genres").agg(avg("Rating").alias("popularity_genres")).filter(col("popularity_genres").isNotNull()).sort(desc("popularity_genres")).show(5)

+-----------+------------------+
 Genres| popularity_genres|
+-----------+------------------+
 Adventure| 4.140160488236416|
 Film-Noir| 4.058143276237343|
Documentary|3.8515663616324893|
 War|3.8347471983177885|
 IMAX| 3.825|
+-----------+------------------+
only showing top 5 rows

In [0]:
#How many movies have been ranked the most consecutive days?
from pyspark.sql.functions import from_utc_timestamp
joinDF= joinDF.withColumn("Timestamp",(col("Timestamp") / 1e6).cast("Timestamp"))



In [0]:
joinDF.show()

+-------+--------------------+---------+------+------+--------------------+
MovieID| Title| Genres|UserID|Rating| Timestamp|
+-------+--------------------+---------+------+------+--------------------+
 122| Boomerang (1992)| Romance| 1| 5|1970-01-01 00:13:...|
 122| Boomerang (1992)| Comedy| 1| 5|1970-01-01 00:13:...|
 185| Net, The (1995)| Thriller| 1| 5|1970-01-01 00:13:...|
 185| Net, The (1995)| Crime| 1| 5|1970-01-01 00:13:...|
 185| Net, The (1995)| Action| 1| 5|1970-01-01 00:13:...|
 231|Dumb & Dumber (1994)| Comedy| 1| 5|1970-01-01 00:13:...|
 292| Outbreak (1995)| Thriller| 1| 5|1970-01-01 00:13:...|
 292| Outbreak (1995)| Sci-Fi| 1| 5|1970-01-01 00:13:...|
 292| Outbreak (1995)| Drama| 1| 5|1970-01-01 00:13:...|
 292| Outbreak (1995)| Action| 1| 5|1970-01-01 00:13:...|
 316| Stargate (1994)| Sci-Fi| 1| 5|1970-01-01 00:13:...|
 316| Stargate (1994)|Adventure| 1| 5|1970-01-01 00:13:...|
 316| Stargate (1994)| Action| 1| 5|1970-01-01 00:13:...|
 329|Star Trek: Genera...| Sci-Fi| 1| 5|1970-01-01 00:13:...|
 329|Star Trek: Genera...| Drama| 1| 5|1970-01-01 00:13:...|
 329|Star Trek: Genera...|Adventure| 1| 5|1970-01-01 00:13:...|
 329|Star Trek: Genera...| Action| 1| 5|1970-01-01 00:13:...|
 355|Flintstones, The ...| Fantasy| 1| 5|1970-01-01 00:13:...|
 355|Flintstones, The ...| Comedy| 1| 5|1970-01-01 00:13:...|
 355|Flintstones, The ...| Children| 1| 5|1970-01-01 00:13:...|
+-------+--------------------+---------+------+------+--------------------+
only showing top 20 rows

In [0]:
# Count the total number of ratings in the dataset
numerator = Ratings_DF.select("Rating").count()

# Count the number of distinct userIds and distinct movieIds
num_users = Ratings_DF.select("UserID").distinct().count()
num_movies = Ratings_DF.select("MovieID").distinct().count()

# Set the denominator equal to the number of users multiplied by the number of movies
denominator = num_users * num_movies

# Divide the numerator by the denominator
sparsity = (1.0 - (numerator *1.0)/denominator)*100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is 98.66% empty.

In [0]:
 #Group data by userId, count ratings
userId_ratings = Ratings_DF.groupBy("UserID").count().orderBy('count', ascending=False)
userId_ratings.show()

+------+-----+
UserID|count|
+------+-----+
 59269| 7359|
 67385| 7047|
 14463| 5169|
 68259| 4483|
 27468| 4449|
 19635| 4165|
 3817| 4165|
 63134| 3755|
 58357| 3697|
 27584| 3479|
 6757| 3414|
 56707| 3225|
 19379| 3202|
 7795| 3187|
 8811| 3164|
 30723| 3027|
 30687| 2909|
 31327| 2886|
 30500| 2827|
 47046| 2812|
+------+-----+
only showing top 20 rows

In [0]:
#Group data by userId, count ratings
movieId_ratings = Ratings_DF.groupBy("MovieID").count().orderBy('count', ascending=False)
movieId_ratings.show()

+-------+-----+
MovieId|count|
+-------+-----+
 296|34864|
 356|34457|
 593|33668|
 480|32631|
 318|31126|
 110|29154|
 457|28951|
 589|28948|
 260|28566|
 150|27035|
 592|26996|
 1|26449|
 780|26042|
 590|25912|
 527|25777|
 380|25381|
 1210|25098|
 32|24397|
 50|24037|
 608|23794|
+-------+-----+
only showing top 20 rows

In [0]:
# Import the required functions
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [0]:
 #Create test and train set
(train, test) = Ratings_DF.randomSplit([0.8, 0.2], seed = 1234)

# Create ALS model
als = ALS(userCol="UserID", itemCol="MovieID", ratingCol="Rating", nonnegative = True, implicitPrefs = False, coldStartStrategy="drop")

# Confirm that a model called "als" was created
type(als)

Out[176]: pyspark.ml.recommendation.ALS

In [0]:
# Import the required functions
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [0]:
# Create test and train set
(train, test) = Ratings_DF.randomSplit([0.8, 0.2], seed = 1234)

# Create ALS model
als = ALS(userCol="UserID", itemCol="MovieID", ratingCol="Rating", nonnegative = True, implicitPrefs = False, coldStartStrategy="drop")

# Confirm that a model called "als" was created
type(als)

Out[195]: pyspark.ml.recommendation.ALS

In [0]:
# Import the requisite items
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Add hyperparameters and their respective values to param_grid
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100, 150]) \
            .addGrid(als.regParam, [.01, .05, .1, .15]) \
            .build()
            #             .addGrid(als.maxIter, [5, 50, 100, 200]) \

           
# Define evaluator as RMSE and print length of evaluator
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating", predictionCol="prediction") 
print ("Num models to be tested: ", len(param_grid))

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-4417672089706678> in <module> 
 1 #Training the Model 
 ----> 2 alsModel = als . fit ( train ) #Generating Predictions 
 3 prediction = alsModel . transform ( test ) 
 4 prediction . show ( 10 ) 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 159 return self . copy ( params ) . _fit ( dataset ) 
 160 else : 
 --> 161 return self . _fit ( dataset ) 
 162 else : 
 163 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit (self, dataset) 
 333 
 334 def _fit ( self , dataset ) : 
 --> 335 java_model = self . _fit_java ( dataset ) 
 336 model = self . _create_model ( java_model ) 
 337 return self . _copyValues ( model ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit_java (self, dataset) 
 330 """
 331 self . _transfer_params_to_java ( ) 
 --> 332 return self . _java_obj . fit ( dataset . _jdf ) 
 333 
 334 def _fit ( self , dataset ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1302 
 1303 answer = self . gateway_client . send_command ( command ) 
 -> 1304 return_value = get_return_value(
 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 108 def deco ( * a , ** kw ) : 
 109 try : 
 --> 110 return f ( * a , ** kw ) 
 111 except py4j . protocol . Py4JJavaError as e : 
 112 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 324 value = OUTPUT_CONVERTER [ type ] ( answer [ 2 : ] , gateway_client ) 
 325 if answer [ 1 ] == REFERENCE_TYPE : 
 --> 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 328 format(target_id, ".", name), value)

 Py4JJavaError : An error occurred while calling o19565.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 240.0 failed 1 times, most recent failure: Lost task 0.0 in stage 240.0 (TID 461) (ip-10-172-228-150.us-west-2.compute.internal executor driver): org.apache.spark.SparkException: Failed to execute user defined function(ALSModelParams$$Lambda$6453/1834507155: (int) => int)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:754)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$SliceIterator.hasNext(Iterator.scala:266)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.generic.Growable.$plus$plus$eq(Growable.scala:62)
	at scala.collection.generic.Growable.$plus$plus$eq$(Growable.scala:53)
	at scala.collection.mutable.ArrayBuffer.$plus$plus$eq(ArrayBuffer.scala:105)
	at scala.collection.mutable.ArrayBuffer.$plus$plus$eq(ArrayBuffer.scala:49)
	at scala.collection.TraversableOnce.to(TraversableOnce.scala:315)
	at scala.collection.TraversableOnce.to$(TraversableOnce.scala:313)
	at scala.collection.AbstractIterator.to(Iterator.scala:1429)
	at scala.collection.TraversableOnce.toBuffer(TraversableOnce.scala:307)
	at scala.collection.TraversableOnce.toBuffer$(TraversableOnce.scala:307)
	at scala.collection.AbstractIterator.toBuffer(Iterator.scala:1429)
	at scala.collection.TraversableOnce.toArray(TraversableOnce.s

In [0]:
# Build cross validation using CrossValidator
cv = CrossValidator(estimator=als, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=5)

# Confirm cv was built
print(cv)

In [0]:
#Fit cross validator to the 'train' dataset
model = cv.fit(train)

#Extract best model from the cv model above
best_model = model.bestModel

In [0]:
# Print best_model
print(type(best_model))

# Complete the code below to extract the ALS model parameters
print("**Best Model**")

# # Print "Rank"
print("  Rank:", best_model._java_obj.parent().getRank())

# Print "MaxIter"
print("  MaxIter:", best_model._java_obj.parent().getMaxIter())

# Print "RegParam"
print("  RegParam:", best_model._java_obj.parent().getRegParam())

In [0]:
# View the predictions
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

In [0]:
# Generate n Recommendations for all users
nrecommendations = best_model.recommendForAllUsers(5)
nrecommendations.limit(10).show()

In [0]:
nrecommendations = nrecommendations\
    .withColumn("rec_exp", explode("recommendations"))\
    .select('userId', col("rec_exp.movieId"), col("rec_exp.rating"))

nrecommendations.limit(10).show()